In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = '~/Documents/DSE/Data/HW3_220/Titanic/'
df = pd.read_csv(path+'train.csv')

Looking at our dataframe

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We do not need Cabin and Ticket and hence can be dropped from our DataFrame.

We also can combine SibSp and Parch to FamilySize.

We can also replace Name with NameLength as richer people tended to have longer names.

In [4]:
df['FamilySize'] = df['SibSp'] + df['Parch']
df['NameLength'] = df['Name'].apply(lambda x: len(x))
df = df.drop(['Ticket', 'Cabin', 'SibSp', 'Parch', 'Name'], axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,NameLength
0,1,0,3,male,22.0,7.2500,S,1,23
1,2,1,1,female,38.0,71.2833,C,1,51
2,3,1,3,female,26.0,7.9250,S,0,22
3,4,1,1,female,35.0,53.1000,S,1,44
4,5,0,3,male,35.0,8.0500,S,0,24


Now let's check for data types and missing values

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            714 non-null float64
Fare           891 non-null float64
Embarked       889 non-null object
FamilySize     891 non-null int64
NameLength     891 non-null int64
dtypes: float64(2), int64(5), object(2)
memory usage: 62.7+ KB


We can see that Age and Embarked has missing data. 

Embarked only has 2 NaNs and can be dropped. Age however has 177 NaNs.

Simply dropping the Age NaNs would mean throwing away too much data. 

We can simply normalize our Age column and replace the NaNs with 0.

In [6]:
#Process Embarked
df = df.dropna(subset=['Embarked'])

#Process Age
df['NewAge']=df['Age']
mean = np.nanmean(df['NewAge'])
std = np.nanstd(df['NewAge'])
df['NewAge']=df['NewAge'].apply(lambda x: (x-mean)/std)
df['NewAge'] = df['NewAge'].fillna(0)
df=df.drop('Age', axis=1)
df.head()

,PassengerId,Survived,Pclass,Sex,Fare,Embarked,FamilySize,NameLength,NewAge
0,1,0,3,male,7.2500,S,1,23,-0.527669
1,2,1,1,female,71.2833,C,1,51,0.577094
2,3,1,3,female,7.9250,S,0,22,-0.251478
3,4,1,1,female,53.1000,S,1,44,0.369951
4,5,0,3,male,8.0500,S,0,24,0.369951


Now we deal with categorical data.

In [7]:
df['Sex']=df['Sex'].map({'male':0, 'female':1})
df['Embarked']=df['Embarked'].map({'S':0, 'C':1, 'Q':2})
df.head()

,PassengerId,Survived,Pclass,Sex,Fare,Embarked,FamilySize,NameLength,NewAge
0,1,0,3,0,7.2500,0,1,23,-0.527669
1,2,1,1,1,71.2833,1,1,51,0.577094
2,3,1,3,1,7.9250,0,0,22,-0.251478
3,4,1,1,1,53.1000,0,1,44,0.369951
4,5,0,3,0,8.0500,0,0,24,0.369951


Finally, we set our PassengerId as our index

In [8]:
df = df.set_index(['PassengerId'])
df.head()

,Survived,Pclass,Sex,Fare,Embarked,FamilySize,NameLength,NewAge
PassengerId,,,,,,,,
1,0,3,0,7.2500,0,1,23,-0.527669
2,1,1,1,71.2833,1,1,51,0.577094
3,1,3,1,7.9250,0,0,22,-0.251478
4,1,1,1,53.1000,0,1,44,0.369951
5,0,3,0,8.0500,0,0,24,0.369951
